In [1]:
%run ../DataClass.ipynb

In [2]:
X_train = np.load('./0/data/X_train.npy')
X_test = np.load('./0/data/X_test.npy')
X_val = np.load('./0/data/X_val.npy')
y_train = np.load('./0/data/y_train.npy')
y_test = np.load('./0/data/y_test.npy')
y_val = np.load('./0/data/y_val.npy')

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from keras.wrappers.scikit_learn import KerasClassifier

# CNN

In [4]:
batch_size = 128
num_classes = 24
epochs = 50

In [5]:
num_labels = num_classes
pixels = int(sqrt(X_train.shape[1]))
# Reshaping to format which CNN expects (batch, height, width, channels)
trainX_cnn = X_train.reshape(X_train.shape[0], pixels, pixels, 1).astype('float32')
valX_cnn = X_val.reshape(X_val.shape[0], pixels, pixels, 1).astype('float32')
testX_cnn= X_test.reshape(X_test.shape[0], pixels, pixels, 1).astype('float32')

# Normalize images from 0-255 to 0-1
trainX_cnn /= 255
valX_cnn /= 255

train_y_cnn = utils.to_categorical(y_train, num_labels)
val_y_cnn = utils.to_categorical(y_val, num_labels)
test_y_cnn = utils.to_categorical(y_test, num_labels)

trainX_cnn, evalX_cnn, train_y_cnn , eval_y_cnn = train_test_split(trainX_cnn, train_y_cnn, test_size=0.2)

In [6]:
from tensorflow.keras.optimizers import SGD


def create_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28 ,1) ))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.20))

    model.add(Dense(24, activation = 'softmax'))

    model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model
# model.save(f'./model_params/CNN_params_tuning')

In [7]:
pixels  = int(sqrt(X_train.shape[1]))
classes = len(np.unique(y_test))

In [8]:
create_model()

In [9]:
model = KerasClassifier(build_fn=create_model,epochs=5, batch_size=128,validation_data=(evalX_cnn, eval_y_cnn))

In [10]:
model.fit(trainX_cnn, train_y_cnn)

Epoch 1/5
130/130 [==============================] - 6s 46ms/step - loss: 2.7271 - accuracy: 0.1765 - val_loss: 1.6616 - val_accuracy: 0.5203
Epoch 2/5
130/130 [==============================] - 6s 45ms/step - loss: 1.3042 - accuracy: 0.5677 - val_loss: 0.8743 - val_accuracy: 0.7153
Epoch 3/5
130/130 [==============================] - 6s 45ms/step - loss: 0.7950 - accuracy: 0.7250 - val_loss: 0.5604 - val_accuracy: 0.8262
Epoch 4/5
130/130 [==============================] - 6s 45ms/step - loss: 0.5654 - accuracy: 0.8054 - val_loss: 0.3675 - val_accuracy: 0.8845
Epoch 5/5
130/130 [==============================] - 6s 46ms/step - loss: 0.3970 - accuracy: 0.8676 - val_loss: 0.2772 - val_accuracy: 0.9107


In [17]:
model_c = CalibratedClassifierCV(base_estimator=model, cv='prefit')


Signature:
keras.models.Model.predict(
    self,
    x,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Source:   
  def predict(self,
              x,
              batch_size=None,
              verbose=0,
              steps=None,
              callbacks=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=False):
    """Generates output predictions for the input samples.

    Computation is done in batches. This method is designed for performance in
    large scale inputs. For small amount of inputs that fit in one batch,
    directly using `__call__` is recommended for faster execution, e.g.,
    `model(x)`, or `model(x, training=False)` if you have layers such as
    `tf.keras.layers.BatchNormalization` that behaves differently during
    inference. Also, note the fact that test loss is not affected by
    regularization layers like noise and

In [14]:
keras.models.Model.predict_proba = keras.models.Model.predict

Signature:
keras.models.Model.predict_proba(
    self,
    x,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Source:   
  def predict(self,
              x,
              batch_size=None,
              verbose=0,
              steps=None,
              callbacks=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=False):
    """Generates output predictions for the input samples.

    Computation is done in batches. This method is designed for performance in
    large scale inputs. For small amount of inputs that fit in one batch,
    directly using `__call__` is recommended for faster execution, e.g.,
    `model(x)`, or `model(x, training=False)` if you have layers such as
    `tf.keras.layers.BatchNormalization` that behaves differently during
    inference. Also, note the fact that test loss is not affected by
    regularization layers like noi

In [21]:
type(valX_cnn),

numpy.ndarray

In [19]:
model_c.fit(valX_cnn, val_y_cnn)

TypeError: predict_proba() missing 1 required positional argument: 'x'

In [18]:
from numpy.random import seed
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [19]:
#load model


# Learn model
# model.fit(trainX_cnn, train_y_cnn, validation_data=(evalX_cnn, eval_y_cnn), epochs=50, batch_size=128)


In [22]:
model.fit(trainX_cnn, train_y_cnn, validation_data=(evalX_cnn, eval_y_cnn), epochs=5, batch_size=128)

NameError: name 'model' is not defined

In [9]:
# model = keras.models.load_model(f'./0/model/CNN_model')

In [10]:
y_pred_val=np.argmax(model.predict(valX_cnn),axis=1)
y_pred_test=np.argmax(model.predict(testX_cnn),axis=1)

In [11]:
print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.8904129367600346
test accuracy: 0.8228414669361825


In [12]:
model 

In [13]:
from keras.wrappers.scikit_learn import KerasClassifier

In [14]:
model_sklearn = KerasClassifier(build_fn=model,
                                 batch_size=128,
                                 epochs=5,
                                 )

In [15]:
model_c = CalibratedClassifierCV(base_estimator=model_sklearn,method='sigmoid', cv='prefit')


In [16]:
model_c.fit(X_val, y_val)

NotFittedError: This KerasClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.